# Artificial Neural Network

### Importing the libraries

In [74]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [75]:
tf.__version__

'2.15.0'

## Part 1 - Data Preprocessing

### Importing the dataset

In [76]:
dataset = pd.read_csv("Churn_Modelling.csv")
X = dataset.iloc[:,3:-1].values
y = dataset.iloc[:,-1].values

In [77]:
X

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [78]:
y

array([1, 0, 1, ..., 1, 1, 0], dtype=int64)

### Encoding categorical data

Label Encoding the "Gender" column

In [79]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

X[:,2] = le.fit_transform(X[:,2])

One Hot Encoding the "Geography" column

In [80]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[1])],
                       remainder='passthrough')
X = np.array(ct.fit_transform(X))
X[0,:]

array([1.0, 0.0, 0.0, 619, 0, 42, 2, 0.0, 1, 1, 1, 101348.88],
      dtype=object)

### Splitting the dataset into the Training set and Test set

In [81]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

### Feature Scaling (Abosultely compulsory for deep learning)

In [82]:
# All the features must be scaled in deep learning

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test)

## Part 2 - Building the ANN

### Initializing the ANN

In [83]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [84]:
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

### Adding the second hidden layer

In [85]:
ann.add(tf.keras.layers.Dense(units=6,activation='sigmoid'))

### Adding the output layer

In [86]:
ann.add(tf.keras.layers.Dense(units=1))

## Part 3 - Training the ANN

### Compiling the ANN

In [87]:
# Providing an optimizer (updates the weights based on the losses), loss function (calculates the loss bw the actual and predicted values)
# 
ann.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

### Training the ANN on the Training set

In [90]:
ann.fit(X_train,y_train,batch_size=32,epochs=100)

Epoch 1/100
250/250 [==============================] - 0s 947us/step - loss: 0.3439 - accuracy: 0.8590
Epoch 2/100
250/250 [==============================] - 0s 926us/step - loss: 0.3431 - accuracy: 0.8587
Epoch 3/100
250/250 [==============================] - 0s 900us/step - loss: 0.3448 - accuracy: 0.8591
Epoch 4/100
250/250 [==============================] - 0s 892us/step - loss: 0.3505 - accuracy: 0.8587
Epoch 5/100
250/250 [==============================] - 0s 952us/step - loss: 0.3450 - accuracy: 0.8590
Epoch 6/100
250/250 [==============================] - 0s 908us/step - loss: 0.3443 - accuracy: 0.8599
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3453 - accuracy: 0.8593
Epoch 8/100
250/250 [==============================] - 0s 944us/step - loss: 0.3474 - accuracy: 0.8577
Epoch 9/100
250/250 [==============================] - 0s 916us/step - loss: 0.3422 - accuracy: 0.8602
Epoch 10/100
250/250 [==============================] - 0s 919us/step - los

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

**Homework**

Use our ANN model to predict if the customer with the following informations will leave the bank: 

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card ? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer ?

**Solution**

Therefore, our ANN model predicts that this customer stays in the bank!

**Important note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

**Important note 2:** Notice also that the "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.

In [106]:
X_test_single = [[1,0,0,600,1,40,3,60000,2,1,1,50000]]
X_test_single = scaler.transform(X_test_single)
if ann.predict(X_test_single) > 0.5:
    print("Customer will stay")
else:
    print("Customer won't stay")

1/1 [==============================] - 0s 20ms/step
Customer won't stay


### Predicting the Test set results

In [113]:
y_pred = (ann.predict(X_test) > 0.5).astype(int)
print(np.concatenate((y_pred.reshape(len(y_pred),1),y_test.reshape(len(y_test),1)),1))

63/63 [==============================] - 0s 884us/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Making the Confusion Matrix

In [114]:
from sklearn.metrics import confusion_matrix, accuracy_score

# Assuming you have predicted labels 'y_pred' and actual labels 'y_true'
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)

print("Confusion Matrix:")
print(cm)
print("Accuracy:", accuracy)

Confusion Matrix:
[[1537   58]
 [ 209  196]]
Accuracy: 0.8665
